In [4]:
import numpy as np


import numpy as np
from scipy.spatial.distance import pdist, squareform

def calculate_distances(array):
    """
    Calculate pairwise distances between all points in a 2D array excluding distances of points with themselves.

    Parameters:
    - array: 2D NumPy array

    Returns:
    - distances: 2D NumPy array containing pairwise distances (excluding distances of points with themselves)
    """
    # Reshape the 2D array to a 1D array of points
    points = array.reshape(-1, 2)

    # Use pdist to calculate pairwise distances
    distances_flat = pdist(points)

    # Convert the flat distances to a square matrix
    distances = squareform(distances_flat)

    # Set distances of points with themselves to a placeholder value (e.g., np.inf)
    np.fill_diagonal(distances, np.inf)

    return distances

# Example usage:
array = np.array([[0, 0], [1, 1], [2, 2]])

distances = calculate_distances(array)
print(distances)

[[       inf 1.41421356 2.82842712]
 [1.41421356        inf 1.41421356]
 [2.82842712 1.41421356        inf]]


In [1]:
import math

def compute_movement(current_pos, obj):
    # Extracting coordinates from current_pos and obj
    current_x, current_y = current_pos
    obj_x, obj_y = obj

    # Calculate distance between current position and the target object
    distance = math.sqrt((obj_x - current_x)**2 + (obj_y - current_y)**2)

    # Calculate angle in radians
    angle_radians = math.atan2(obj_y - current_y, obj_x - current_x)

    # Convert angle to degrees
    angle_degrees = math.degrees(angle_radians)

    
    print(f"Distance: {distance}", f"degrees: {angle_degrees}")
    return distance, angle_degrees
    

# Example usage
compute_movement([0, 0], [1, -6])
    

    

Distance: 6.082762530298219 degrees: -80.53767779197439


(6.082762530298219, -80.53767779197439)

In [31]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
import time
import numpy as np
SPEED = 50
#--- one cell
import asyncio
thymio_speed_to_mms = 21.73913043478261/50 ##TO FIND THE MM/s speed, divide motors_speed by this, thix value has to be tuned for each thymio
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(SPEED)],
        "motor.right.target": [int(SPEED)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)

    # Stop the motors
    v_stop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(v_stop)
async def rotate_for_seconds(client, seconds, degrees):
    if degrees >=0:
        v = {
            "motor.left.target": [int(SPEED)],
            "motor.right.target": [-int(SPEED)],
        }
        await node.set_variables(v)
        await asyncio.sleep(seconds)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        await node.set_variables(v_stop)
    else:
        v = {
            "motor.left.target": [-int(SPEED)],
            "motor.right.target": [int(SPEED)],
        }
        await node.set_variables(v)
        await asyncio.sleep(seconds)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        await node.set_variables(v_stop)

    
async def print_elapsed_time(seconds, interval=0.5):
    start_time = time.monotonic()
    while True:
        elapsed = time.monotonic() - start_time
        if elapsed > seconds:
            break
        print(f"Elapsed time: {elapsed:.4f} seconds")
        await asyncio.sleep(interval)

async def main():    
    # Start the driving and printing tasks
    
    dist, angle = compute_movement([0,0], [10,10])
    secondstodrive = dist/(SPEED*thymio_speed_to_mms)# dist*thymio_speed_to_mms/SPEED
    print(f"seconds :", secondstodrive)
    #secondstorotate = SPEED*thymio_speed_to_mms/angle
    if angle != 0:
        
        disttorotate = angle*12.002/253.3
        secondstorotate = disttorotate/(SPEED*thymio_speed_to_mms)
        rotate_task = asyncio.create_task(rotate_for_seconds(client, disttorotate, np.sign(angle)))
        await rotate_task
    drive_task = asyncio.create_task(drive_for_seconds(client, secondstodrive))
    
    print_task = asyncio.create_task(print_elapsed_time(secondstodrive))
    
    # Wait for both tasks to complete
    await drive_task
    
    await print_task
    
await main()
await node.unlock()

Distance: 14.142135623730951 degrees: 45.0
seconds : 0.6505382386916237
Elapsed time: 0.0000 seconds
Elapsed time: 0.5150 seconds


Quick explaination on how we computed the seconds required to rotate of x degrees


You can use the following formula to calculate the number of rotations \(T\) required for one complete rotation:
$$T = \frac{L}{\pi \times D} $$
We measured L = 9.5cm (distance between wheels) and D = 4.3 cm (wheel diameter)
we want to know the number of degrees the wheels will turn for the vehicle to complete one full rotation, we can use the formula:
$$\text{Number of degrees} = T \times 360 $$

Substitute the value of \(T\) from the previous formula:
$$\text{Number of degrees} = \frac{L}{\pi \times D} \times 360 = 253.2958$$

Then from this we can compute the required distance to travel for a given wanted rotation: 
$$\frac{253.3}{4.3*\pi} = \frac{wanted rotation°}{distance} \implies distance = \frac{\text{wanted rotation°} \times 4.3 \pi}{253.3}$$
From which we compute the seconds required:
$$\text{Seconds of rotation} =  \frac{distance}{SPEED \times conversionfactor}$$
